## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

## Data Pre-Processing

### Loading Datasets

In [2]:
training_data = pd.read_csv('data_format1/data_format1/train_format1.csv')
testing_data = pd.read_csv('data_format1/data_format1/test_format1.csv')
user_demo = pd.read_csv('data_format1/data_format1/user_info_format1.csv')
user_logs = pd.read_csv('data_format1/data_format1/user_log_format1.csv')
testing_data.drop('prob', axis=1, inplace=True)

### Data Cleaning

In [3]:
# Check if any column in the tables contains any empty data
print(user_demo.columns[user_demo.isnull().any()].tolist())
print(user_logs.columns[user_logs.isnull().any()].tolist())

['age_range', 'gender']
['brand_id']


In [4]:
user_logs['user_id'] = user_logs['user_id'].astype(np.int32)
user_logs['item_id'] = user_logs['item_id'].astype(np.int32)
user_logs['cat_id'] = user_logs['cat_id'].astype(np.int16)
user_logs['seller_id'] = user_logs['seller_id'].astype(np.int16)
user_logs.rename(columns={'seller_id' : 'merchant_id'}, inplace=True)
user_logs['brand_id'].fillna(0, inplace=True)
user_logs['brand_id'] = user_logs['brand_id'].astype(np.int16)
user_logs['time_stamp'] = (pd.to_datetime(user_logs['time_stamp'], format='%m%d') - pd.to_datetime(user_logs['time_stamp'].min(), format='%m%d')).dt.days
user_logs['time_stamp'] = user_logs['time_stamp'].astype(np.int16)
user_logs['action_type'] = user_logs['action_type'].astype(np.int8)

C:\Users\chuas\AppData\Local\Temp\ipykernel_1640\3035380006.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_logs['brand_id'].fillna(0, inplace=True)


In [5]:
user_demo['age_range'].fillna(0, inplace=True)
user_demo['gender'].fillna(2, inplace=True)
user_demo['age_range'] = user_demo['age_range'].astype(np.int8)
user_demo['gender'] = user_demo['gender'].astype(np.int8)
user_logs['time_period'] = user_logs['time_stamp'] // 31

C:\Users\chuas\AppData\Local\Temp\ipykernel_1640\2765511242.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_demo['age_range'].fillna(0, inplace=True)
C:\Users\chuas\AppData\Local\Temp\ipykernel_1640\2765511242.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [ ]:
df_train = training_data.merge(user_demo, on='user_id', how='left')
df_test = testing_data.merge(user_demo, on='user_id', how='left')

df_train = df_train.merge(
    user_logs[['user_id', 'merchant_id', 'item_id', 'cat_id', 'brand_id','time_stamp','time_period']].drop_duplicates(),
    on=['user_id', 'merchant_id'],
    how='left'
)

df_test = df_test.merge(
    user_logs[['user_id', 'merchant_id', 'item_id', 'cat_id', 'brand_id','time_stamp','time_period']].drop_duplicates(),
    on=['user_id', 'merchant_id'],
    how='left'
)

df_train['type'] = 'train'
df_test['type'] = 'test'
df = pd.concat([df_train, df_test], ignore_index=True)

## Feature Engineering

In [7]:
# Grouping for easy aggregration
users = user_logs.groupby('user_id')
merchants = user_logs.groupby('merchant_id')
brands = user_logs.groupby('brand_id')
categories = user_logs.groupby('cat_id')
items = user_logs.groupby('item_id')
users_merchants = user_logs.groupby(['user_id', 'merchant_id'])
users_brands = user_logs.groupby(['user_id', 'brand_id'])
users_categories = user_logs.groupby(['user_id', 'cat_id'])
merchant_brands = user_logs.groupby(['merchant_id', 'brand_id'])
merchant_categories = user_logs.groupby(['merchant_id', 'cat_id'])
user_logs['time_period'] = user_logs['time_stamp'] // 31

In [8]:
# One-hot encoding for the age_range column
age_dummies = pd.get_dummies(user_demo[['user_id', 'age_range']], columns=['age_range'], prefix='age')

df = df.merge(age_dummies, on='user_id', how='left')
df_train = df_train.merge(age_dummies, on='user_id', how='left')
df.drop('age_range', axis=1, inplace=True)
df_train.drop('age_range', axis=1, inplace=True)

### Overall Count and Action Count/Ratio Features

### Overall Count of Unique Values

In [9]:
# Count total number of unique values for a given user
to_merge = users.nunique().reset_index().rename(columns={
    'item_id': 'items_user', 
    'cat_id': 'categories_user',
    'merchant_id': 'merchants_user',
    'brand_id': 'brands_user',
    'time_stamp': 'dates_user',
    'time_period': 'periods_user',
    'action_type': 'action_types_user'
})
df = df.merge(to_merge, on='user_id', how='left')

# Count total number of unique values for a given merchant
to_merge = merchants.nunique().reset_index().rename(columns={
    'item_id': 'items_merchant', 
    'cat_id': 'categories_merchant',
    'user_id': 'users_merchant',
    'brand_id': 'brands_merchant',
    'time_stamp': 'dates_merchant',
    'time_period': 'periods_merchant',
    'action_type': 'action_types_merchant'
})
df = df.merge(to_merge, on='merchant_id', how='left')

# Count total number of unique values for a given user and merchant
to_merge = users_merchants.nunique().reset_index().rename(columns={
    'item_id': 'items_user_merchant', 
    'cat_id': 'categories_user_merchant',
    'brand_id': 'brands_user_merchant',
    'time_stamp': 'dates_user_merchant',
    'time_period': 'periods_user_merchant',
    'action_type': 'action_types_user_merchant'
})
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')

# Count total number of unique values for a given item
to_merge = items.nunique().reset_index().rename(columns={
    'user_id': 'users_item',
    'cat_id': 'categories_item',
    'merchant_id': 'merchants_item',
    'brand_id': 'brands_item',
    'time_stamp': 'dates_item',
    'time_period': 'periods_item',
    'action_type': 'action_types_item'
})
df = df.merge(to_merge, on='item_id', how='left')

# Count total number of unique values for a given category
to_merge = categories.nunique().reset_index().rename(columns={
    'item_id': 'items_category',
    'user_id': 'users_category',
    'merchant_id': 'merchants_category',
    'brand_id': 'brands_category',
    'time_stamp': 'dates_category',
    'time_period': 'periods_category',
    'action_type': 'action_types_category'
})
df = df.merge(to_merge, on='cat_id', how='left')

# Count total number of unique values for a given brand
to_merge = brands.nunique().reset_index().rename(columns={
    'item_id': 'items_brand',
    'user_id': 'users_brand',
    'merchant_id': 'merchants_brand',
    'cat_id': 'categories_brand',
    'time_stamp': 'dates_brand',
    'time_period': 'periods_brand',
    'action_type': 'action_types_brand'
})
df = df.merge(to_merge, on='brand_id', how='left')

# Count total number of unique values for a given user and brand
to_merge = users_brands.nunique().reset_index().rename(columns={
    'item_id': 'items_user_brand',
    'cat_id': 'categories_user_brand',
    'merchant_id': 'merchants_user_brand',
    'time_stamp': 'dates_user_brand',
    'time_period': 'periods_user_brand',
    'action_type': 'action_types_user_brand'
})
df = df.merge(to_merge, on=['user_id', 'brand_id'], how='left')

# Count total number of unique values for a given user and category
to_merge = users_categories.nunique().reset_index().rename(columns={
    'item_id': 'items_user_category',
    'merchant_id': 'merchants_user_category',
    'brand_id': 'brands_user_category',
    'time_stamp': 'dates_user_category',
    'time_period': 'periods_user_category',
    'action_type': 'action_types_user_category'
})
df = df.merge(to_merge, on=['user_id', 'cat_id'], how='left')

# Count total number of unique values for a given merchant and brand
to_merge = merchant_brands.nunique().reset_index().rename(columns={
    'item_id': 'items_merchant_brand',
    'cat_id': 'categories_merchant_brand',
    'user_id': 'users_merchant_brand',
    'time_stamp': 'dates_merchant_brand',
    'time_period': 'periods_merchant_brand',
    'action_type': 'action_types_merchant_brand'
})
df = df.merge(to_merge, on=['merchant_id', 'brand_id'], how='left')

# Count total number of unique values for a given merchant and category
to_merge = merchant_categories.nunique().reset_index().rename(columns={
    'item_id': 'items_merchant_category',
    'user_id': 'users_merchant_category',
    'brand_id': 'brands_merchant_category',
    'time_stamp': 'dates_merchant_category',
    'time_period': 'periods_merchant_category',
    'action_type': 'action_types_merchant_category'
})
df = df.merge(to_merge, on=['merchant_id', 'cat_id'], how='left')

### Overall Action Count

In [10]:
str_list = ['user', 'merchant', 'brand', 'cat', 'item', 'user_merchant', 'user_brand', 'user_cat', 'merchant_brand', 'merchant_cat']
groups = [users, merchants, brands, categories, items, users_merchants, users_brands, users_categories, merchant_brands, merchant_categories]

for i in range(len(str_list)):
    if i >= 5:
        keys = str_list[i].split('_')
        merge_keys = [f"{keys[0]}_id", f"{keys[1]}_id"]
    else:
        merge_keys = str_list[i] + '_id'
    # Count total actions type for given user
    to_merge = groups[i]['action_type'].value_counts().unstack(fill_value=0).rename(columns={
        0: 'clicks_' + str_list[i], 
        1: 'carts_' + str_list[i], 
        2: 'purchases_' + str_list[i], 
        3: 'favourites_'+ str_list[i]
    })
    df = df.merge(to_merge, on=merge_keys, how='left')

df.head()

,user_id,merchant_id,label,gender,item_id,cat_id,brand_id,time_stamp,time_period,type,...,purchases_user_cat,favourites_user_cat,clicks_merchant_brand,carts_merchant_brand,purchases_merchant_brand,favourites_merchant_brand,clicks_merchant_cat,carts_merchant_cat,purchases_merchant_cat,favourites_merchant_cat
0,34176,3906,0.0,0,757713,821,6268,183,5,train,...,1,1,14870,1,410,961,2790,6,103,207
1,34176,3906,0.0,0,718096,1142,6268,173,5,train,...,0,2,14870,1,410,961,1421,4,33,81
2,34176,3906,0.0,0,757713,821,6268,173,5,train,...,1,1,14870,1,410,961,2790,6,103,207
3,34176,3906,0.0,0,613698,821,6268,163,5,train,...,1,1,14870,1,410,961,2790,6,103,207
4,34176,3906,0.0,0,757713,821,6268,181,5,train,...,1,1,14870,1,410,961,2790,6,103,207


### Overall Action Count Ratio

In [11]:
str_list = ['user', 'merchant', 'brand', 'cat', 'item', 'user_merchant', 'user_brand', 'user_cat', 'merchant_brand', 'merchant_cat']

for profile in str_list:
    # Calculate the total actions for the profile
    df[f'total_actions_{profile}'] = (
        df[f'clicks_{profile}'] +
        df[f'carts_{profile}'] +
        df[f'purchases_{profile}'] +
        df[f'favourites_{profile}']
    )
    
    # Calculate action ratios~
    df[f'clicks_ratio_{profile}'] = df[f'clicks_{profile}'] / (df[f'total_actions_{profile}'] + 1e-9)
    df[f'carts_ratio_{profile}'] = df[f'carts_{profile}'] / (df[f'total_actions_{profile}'] + 1e-9)
    df[f'purchases_ratio_{profile}'] = df[f'purchases_{profile}'] / (df[f'total_actions_{profile}'] + 1e-9)
    df[f'favourites_ratio_{profile}'] = df[f'favourites_{profile}'] / (df[f'total_actions_{profile}'] + 1e-9)


### Overall Day Count

In [12]:
str_list = ['user', 'merchant', 'brand', 'cat', 'item', 'user_merchant', 'user_brand', 'user_cat', 'merchant_brand', 'merchant_cat']
groups = [users, merchants, brands, categories, items, users_merchants, users_brands, users_categories, merchant_brands, merchant_categories]

for i in range(len(str_list)):
    if i >= 5: 
        keys = str_list[i].split('_')
        merge_keys = [f"{keys[0]}_id", f"{keys[1]}_id", 'time_stamp']
        group_by_keys = [f"{keys[0]}_id", f"{keys[1]}_id", 'time_stamp', 'action_type']
    else: 
        merge_keys = [f'{str_list[i]}_id', 'time_stamp']
        group_by_keys = [f'{str_list[i]}_id', 'time_stamp', 'action_type']
    
    # Group by entity (or pair of entities), time_stamp, and action_type
    to_merge = (
        user_logs.groupby(group_by_keys).size().unstack(level=-1, fill_value=0).reset_index().rename(columns={
            0: f'clicks_{str_list[i]}_per_day', 
            1: f'carts_{str_list[i]}_per_day', 
            2: f'purchases_{str_list[i]}_per_day', 
            3: f'favourites_{str_list[i]}_per_day'
        })
    )

    df = df.merge(to_merge, on=merge_keys, how='left')


### Monthly Action Count

In [13]:
str_list = ['user', 'merchant', 'brand', 'cat', 'item', 'user_merchant']
groups = [users, merchants, brands, categories, items, users_merchants]

for i in range(len(str_list)):
    if i >= 5: 
        keys = str_list[i].split('_')  # Split for two-entity relationships
        merge_keys = [f"{keys[0]}_id", f"{keys[1]}_id", 'time_period']
        group_by_keys = [f"{keys[0]}_id", f"{keys[1]}_id", 'time_period', 'action_type']
    else: 
        merge_keys = [f'{str_list[i]}_id', 'time_period']
        group_by_keys = [f'{str_list[i]}_id', 'time_period', 'action_type']
    
    # Group by entity (or pair of entities), time_period, and action_type
    to_merge = (
        user_logs.groupby(group_by_keys)
        .size()
        .unstack(level=-1, fill_value=0)
        .reset_index()
        .rename(columns={
            0: f'clicks_{str_list[i]}_monthly', 
            1: f'carts_{str_list[i]}_monthly', 
            2: f'purchases_{str_list[i]}_monthly', 
            3: f'favourites_{str_list[i]}_monthly'
        })
    )

    df = df.merge(to_merge, on=merge_keys, how='left')

### Monthly Action Count Ratio

In [14]:
str_list = ['user', 'merchant', 'brand', 'cat', 'item', 'user_merchant']

for profile in str_list:
    df[f'total_actions_{profile}_monthly'] = (
        df[f'clicks_{profile}_monthly'] +
        df[f'carts_{profile}_monthly'] +
        df[f'purchases_{profile}_monthly'] +
        df[f'favourites_{profile}_monthly']
    )
    
    df[f'clicks_ratio_{profile}_monthly'] = df[f'clicks_{profile}_monthly'] / (df[f'total_actions_{profile}_monthly'] + 1e-9)
    df[f'carts_ratio_{profile}_monthly'] = df[f'carts_{profile}_monthly'] / (df[f'total_actions_{profile}_monthly'] + 1e-9)
    df[f'purchases_ratio_{profile}_monthly'] = df[f'purchases_{profile}_monthly'] / (df[f'total_actions_{profile}_monthly'] + 1e-9)
    df[f'favourites_ratio_{profile}_monthly'] = df[f'favourites_{profile}_monthly'] / (df[f'total_actions_{profile}_monthly'] + 1e-9)

### Product Diversity

In [15]:
profiles = [
    {'name': 'user', 'keys': ['user_id', 'time_period']},
    {'name': 'merchant', 'keys': ['merchant_id', 'time_period']},
    {'name': 'user_merchant', 'keys': ['user_id', 'merchant_id', 'time_period']}
]

diversity_entities = ['item_id', 'brand_id', 'cat_id']

for profile in profiles:
    for entity in diversity_entities:
        diversity_features = (
            user_logs.groupby(profile['keys'] + ['action_type'])[entity]
            .nunique()
            .unstack(level=-1, fill_value=0)
            .reset_index()
            .rename(columns={
                0: f'unique_{entity}_clicks_{profile["name"]}_monthly',
                1: f'unique_{entity}_carts_{profile["name"]}_monthly',
                2: f'unique_{entity}_purchases_{profile["name"]}_monthly',
                3: f'unique_{entity}_favourites_{profile["name"]}_monthly'
            })
        )

        df = df.merge(diversity_features, on=profile['keys'], how='left')


### Penetration

purchase_penetration:
Number of unique users who purchased the given entity within a time period.

interaction_penetration:
Proportion of unique users interacting with the entity relative to the total unique users.


In [16]:
entities = ['item_id', 'merchant_id', 'brand_id', 'cat_id']

for entity in entities:
    # PURCHASE-BASED PENETRATION
    purchase_penetration = (
        user_logs[user_logs['action_type'] == 2]  # Filter for purchases
        .groupby([entity, 'time_period'])['user_id']
        .nunique()
        .reset_index()
        .rename(columns={'user_id': f'{entity}_purchase_penetration'})
    ).fillna(0)

    # INTERACTION-BASED PENETRATION
    total_users = user_logs['user_id'].nunique()  # Total unique users
    interaction_penetration = (
        user_logs.groupby([entity, 'time_period'])['user_id']
        .nunique()  
        .reset_index()
        .rename(columns={'user_id': f'{entity}_interaction_penetration'})
    )
    interaction_penetration[f'{entity}_interaction_penetration'] /= total_users  # Normalize
    
    df = df.merge(purchase_penetration, on=[entity, 'time_period'], how='left')
    df = df.merge(interaction_penetration, on=[entity, 'time_period'], how='left')

### Aggregation Features

### User Aggregation

In [17]:
aggregations = ['mean', 'std', 'max', 'median']
entities = ['merchant_id', 'brand_id', 'cat_id', 'user_merchant']

action_types = {0: 'clicks', 3: 'favourites'}

purchase_log = user_logs[user_logs['action_type'] == 2]

for entity in entities:
    if entity == 'user_merchant':
        group_keys = ['user_id', 'merchant_id']
        aggregate_keys = ['merchant_id']
    else:
        group_keys = ['user_id', entity]
        aggregate_keys = [entity]

    # USER-PURCHASE-DAY AGGREGATION
    user_purchase_day = (
        purchase_log.groupby(group_keys)['time_stamp'].nunique().reset_index().rename(columns={
            'time_stamp': f'user_purchase_days_{entity}'
            })
    ).fillna(0)
    
    # USER-PURCHASE-ITEM AGGREGATION
    user_purchase_item = (
        purchase_log.groupby(group_keys)['item_id'].nunique().reset_index().rename(columns={
            'item_id': f'user_purchase_items_{entity}'
            })
    ).fillna(0)

    purchase_day_aggregations = (
        user_purchase_day.groupby(aggregate_keys)[f'user_purchase_days_{entity}']
        .agg(aggregations)
        .reset_index()
        .rename(columns={metric: f'user_purchase_days_{entity}_{metric}' for metric in aggregations})
    )

    purchase_item_aggregations = (
        user_purchase_item.groupby(aggregate_keys)[f'user_purchase_items_{entity}']
        .agg(aggregations)
        .reset_index()
        .rename(columns={metric: f'user_purchase_items_{entity}_{metric}' for metric in aggregations})
    )

    df = df.merge(purchase_day_aggregations, on=aggregate_keys, how='left')
    df = df.merge(purchase_item_aggregations, on=aggregate_keys, how='left')

    # Calculate features for clicks and favourites for merchants only
    if entity == 'merchant_id':
        for action, action_name in action_types.items():
            user_action_day = (
                user_logs[user_logs['action_type'] == action]
                .groupby(group_keys)['time_stamp'].nunique()
                .reset_index()
                .rename(columns={'time_stamp': f'user_{action_name}_days_{entity}'})
            ).fillna(0)

            user_action_item = (
                user_logs[user_logs['action_type'] == action]
                .groupby(group_keys)['item_id'].nunique()
                .reset_index()
                .rename(columns={'item_id': f'user_{action_name}_items_{entity}'})
            ).fillna(0)

            action_day_aggregations = (
                user_action_day.groupby(aggregate_keys)[f'user_{action_name}_days_{entity}']
                .agg(aggregations)
                .reset_index()
                .rename(columns={metric: f'user_{action_name}_days_{entity}_{metric}' for metric in aggregations})
            )

            action_item_aggregations = (
                user_action_item.groupby(aggregate_keys)[f'user_{action_name}_items_{entity}']
                .agg(aggregations)
                .reset_index()
                .rename(columns={metric: f'user_{action_name}_items_{entity}_{metric}' for metric in aggregations})
            )

            df = df.merge(action_day_aggregations, on=aggregate_keys, how='left')
            df = df.merge(action_item_aggregations, on=aggregate_keys, how='left')

### Merchant Aggregation

In [18]:
metrics = ['mean', 'std', 'max', 'median']

merchant_aggregation_features = {}

merchant_purchase_day = (
    user_logs[user_logs['action_type'] == 2]  
    .groupby(['user_id', 'merchant_id'])['time_stamp']
    .nunique()
    .reset_index()
    .rename(columns={'time_stamp': 'merchant_purchase_days'})
)

merchant_purchase_day_agg = (
    merchant_purchase_day.groupby('user_id')['merchant_purchase_days']
    .agg(metrics)
    .reset_index()
    .rename(columns={metric: f'merchant_purchase_days_{metric}' for metric in metrics})
)
merchant_aggregation_features['purchase_day'] = merchant_purchase_day_agg

merchant_purchase_item = (
    user_logs[user_logs['action_type'] == 2] 
    .groupby(['user_id', 'merchant_id'])['item_id']
    .nunique()
    .reset_index()
    .rename(columns={'item_id': 'merchant_purchase_items'})
)

merchant_purchase_item_agg = (
    merchant_purchase_item.groupby('user_id')['merchant_purchase_items']
    .agg(metrics)
    .reset_index()
    .rename(columns={metric: f'merchant_purchase_items_{metric}' for metric in metrics})
)
merchant_aggregation_features['purchase_item'] = merchant_purchase_item_agg

action_types = {0: 'clicks', 1: 'carts', 2: 'purchases', 3: 'favourites'}

for action, action_name in action_types.items():
    merchant_action_day = (
        user_logs[user_logs['action_type'] == action]
        .groupby(['user_id', 'merchant_id'])['time_stamp']
        .nunique()
        .reset_index()
        .rename(columns={'time_stamp': f'merchant_{action_name}_days'})
    )

    merchant_action_day_agg = (
        merchant_action_day.groupby('user_id')[f'merchant_{action_name}_days']
        .agg(metrics)
        .reset_index()
        .rename(columns={metric: f'merchant_{action_name}_days_{metric}' for metric in metrics})
    )
    merchant_aggregation_features[f'{action_name}_day'] = merchant_action_day_agg

for feature_name, feature_df in merchant_aggregation_features.items():
    df = df.merge(feature_df, on='user_id', how='left')



### Double 11 Features

In [19]:
DOUBLE_11= 184 # min date is 511
double_11_log = (user_logs[user_logs['time_stamp'] ==DOUBLE_11]).reset_index(drop=True)

double_11_users= double_11_log.groupby('user_id')
double_11_merchant = double_11_log.groupby('merchant_id')
ouble_11_categories = double_11_log.groupby('cat_id')
double_11_brand = double_11_log.groupby('brand_id')
double_11_item = double_11_log.groupby('item_id')
double_11_user_merchant = double_11_log.groupby(['user_id','merchant_id'])
# counts of clicks, purchase, addto favourite

# Count each action type for each user
# count total number of unique values from each feature for a given user 
# count total number of unique values from each feature for a given user 
to_merge = double_11_users.nunique().reset_index().rename(columns={
    'item_id': 'double_11_items_user', 
    'cat_id': 'double_11_categories_user',
    'merchant_id': 'double_11_merchants_user',
    'brand_id': 'double_11_brands_user',
    'time_stamp': 'double_11_dates_user',
    'time_period': 'double_11_periods_user',
    'action_type': 'double_11_action_types_user'
    })

df = df.merge(to_merge, on='user_id', how='left')

# count total number of unique values from each feature for a given merchant 
to_merge = double_11_merchant.nunique().reset_index().rename(columns={
    'item_id': 'double_11_items_merchant', 
    'cat_id': 'double_11_categories_merchant',
    'user_id': 'double_11_users_merchant',
    'brand_id': 'double_11_brands_merchant',
    'time_stamp': 'double_11_dates_merchant',
    'time_period': 'double_11_periods_merchant',
    'action_type': 'double_11_action_types_merchant'
    })
df = df.merge(to_merge, on='merchant_id', how='left')

# count total number of unique values from each feature for a given user and merchant
to_merge = double_11_user_merchant.nunique().reset_index().rename(columns={
    'item_id': 'double_11_items_user_merchant', 
    'cat_id': 'double_11_categories_user_merchant',
    'brand_id': 'double_11_brands_user_merchant',
    'time_stamp': 'double_11_dates_user_merchant',
    'time_period': 'double_11_periods_user_merchant',
    'action_type': 'double_11_action_types_user_merchant'
    })
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')

# count total actions by type for a given user
to_merge = double_11_users['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'double_11_clicks_user',
    1: 'double_11_carts_user',
    2: 'double_11_purchases_user',
    3: 'double_11_favourites_user'
    })
df = df.merge(to_merge, on='user_id', how='left')

# count total actions by type for a given merchant
to_merge = double_11_merchant['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'double_11_clicks_merchant', 
    1: 'double_11_carts_merchant',
    2: 'double_11_purchases_merchant',
    3: 'double_11_favourites_merchant'
    })
df = df.merge(to_merge, on='merchant_id', how='left')

# count total actions by type for a given pair (user, merchant)
to_merge = double_11_user_merchant['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'double_11_clicks_user_merchant',
    1: 'double_11_carts_user_merchant',
    2: 'double_11_purchases_user_merchant',
    3: 'double_11_favourites_user_merchant'
    })
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')


### Ratio

In [20]:
groups = ['user', 'merchant', 'user_merchant']
action_types = ['clicks', 'carts', 'purchases', 'favourites']

EPSILON = 1e-8

# Compute ratios for features in user, merchant, and user-merchant categories
for group in groups:
    for feature in ['items', 'categories', 'brands', 'dates', 'periods', 'action_types']:
        # Feature ratio for each group relative to itself
        df[f'double_11_{feature}_{group}_ratio'] = df[f'double_11_{feature}_{group}'] / (df[f'double_11_{feature}_{group}'] + EPSILON)
        df[f'double_11_{feature}_{group}_ratio'] = df[f'double_11_{feature}_{group}'] / (df[f'{feature}_{group}'] + EPSILON)

# Compute ratios for actions from the user perspective
for action in action_types:
    # Ratio within double_11 actions
    df[f'double_11_{action}_user_ratio'] = df[f'double_11_{action}_user'] / (
        df['double_11_clicks_user'] + df['double_11_carts_user'] + df['double_11_purchases_user'] + df['double_11_favourites_user'] + EPSILON
    )
    df[f'double_11_{action}_user_ratio'] = df[f'double_11_{action}_user'] / (df[f'{action}_user'] + EPSILON)

# Compute ratios for actions from the merchant perspective
for action in action_types:
    # Ratio within double_11 actions
    df[f'double_11_{action}_merchant_ratio'] = df[f'double_11_{action}_merchant'] / (
        df['double_11_clicks_merchant'] + df['double_11_carts_merchant'] + df['double_11_purchases_merchant'] + df['double_11_favourites_merchant'] + EPSILON
    )
    df[f'double_11_{action}_merchant_ratio'] = df[f'double_11_{action}_merchant'] / (df[f'{action}_merchant'] + EPSILON)

# Compute ratios for actions from the user-merchant perspective
for action in action_types:
    # Ratio within double_11 actions
    df[f'double_11_{action}_user_merchant_ratio'] = df[f'double_11_{action}_user_merchant'] / (
        df['double_11_clicks_user_merchant'] + df['double_11_carts_user_merchant'] + df['double_11_purchases_user_merchant'] + df['double_11_favourites_user_merchant'] + EPSILON
    )
    # Ratio of actions compared to total user-merchant actions
    df[f'double_11_{action}_user_merchant_ratio'] = df[f'double_11_{action}_user_merchant'] / (df[f'{action}_user_merchant'] + EPSILON)

# Ratio of actions in each merchant (user perspective)
for action in action_types:
    df[f'double_11_{action}_in_merchant_ratio_perspective'] = df[f'double_11_{action}_user_merchant'] / (df[f'double_11_{action}_user'] + EPSILON)

# Ratio of actions in each merchant (merchant perspective)
for action in action_types:
    df[f'double_11_{action}_by_user_ratio_perspective'] = df[f'double_11_{action}_user_merchant'] / (df[f'double_11_{action}_merchant'] + EPSILON)

# Ratio of each action type for a given user
for action in action_types:
    df[f'double_11_{action}_user_ratio'] = df[f'double_11_{action}_user'] / (
        df['double_11_clicks_user'] + df['double_11_carts_user'] + df['double_11_purchases_user'] + df['double_11_favourites_user'] + EPSILON
    )

# Ratio of each action type for a given merchant
for action in action_types:
    df[f'double_11_{action}_merchant_ratio'] = df[f'double_11_{action}_merchant'] / (
        df['double_11_clicks_merchant'] + df['double_11_carts_merchant'] + df['double_11_purchases_merchant'] + df['double_11_favourites_merchant'] + EPSILON
    )

# Ratio of each action type for a given pair (user, merchant)
for action in action_types:
    df[f'double_11_{action}_user_merchant_ratio'] = df[f'double_11_{action}_user_merchant'] / (
        df['double_11_clicks_user_merchant'] + df['double_11_carts_user_merchant'] + df['double_11_purchases_user_merchant'] + df['double_11_favourites_user_merchant'] + EPSILON
    )

### Latest one week

In [21]:
latest_one_week_log = user_logs[
    (user_logs['time_stamp'] >= DOUBLE_11 - 7) & 
    (user_logs['time_stamp'] < DOUBLE_11 )
].reset_index(drop=True)


latest_one_week_users = latest_one_week_log.groupby('user_id')
latest_one_week_merchant = latest_one_week_log.groupby('merchant_id')
latest_one_week_categories = latest_one_week_log.groupby('cat_id')
latest_one_week_brand = latest_one_week_log.groupby('brand_id')
latest_one_week_item = latest_one_week_log.groupby('item_id')
latest_one_week_user_merchant = latest_one_week_log.groupby(['user_id', 'merchant_id'])

# Count each action type for each user
# Count total number of unique values from each feature for a given user
to_merge = latest_one_week_users.nunique().reset_index().rename(columns={
    'item_id': 'latest_one_week_items_user', 
    'cat_id': 'latest_one_week_categories_user',
    'merchant_id': 'latest_one_week_merchants_user',
    'brand_id': 'latest_one_week_brands_user',
    'time_stamp': 'latest_one_week_dates_user',
    'time_period': 'latest_one_week_periods_user',
    'action_type': 'latest_one_week_action_types_user'
})

df = df.merge(to_merge, on='user_id', how='left')

# Count total number of unique values from each feature for a given merchant
to_merge = latest_one_week_merchant.nunique().reset_index().rename(columns={
    'item_id': 'latest_one_week_items_merchant', 
    'cat_id': 'latest_one_week_categories_merchant',
    'user_id': 'latest_one_week_users_merchant',
    'brand_id': 'latest_one_week_brands_merchant',
    'time_stamp': 'latest_one_week_dates_merchant',
    'time_period': 'latest_one_week_periods_merchant',
    'action_type': 'latest_one_week_action_types_merchant'
})
df = df.merge(to_merge, on='merchant_id', how='left')

# Count total number of unique values from each feature for a given user and merchant
to_merge = latest_one_week_user_merchant.nunique().reset_index().rename(columns={
    'item_id': 'latest_one_week_items_user_merchant', 
    'cat_id': 'latest_one_week_categories_user_merchant',
    'brand_id': 'latest_one_week_brands_user_merchant',
    'time_stamp': 'latest_one_week_dates_user_merchant',
    'time_period': 'latest_one_week_periods_user_merchant',
    'action_type': 'latest_one_week_action_types_user_merchant'
})
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')

# Count total actions by type for a given user
to_merge = latest_one_week_users['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'latest_one_week_clicks_user',
    1: 'latest_one_week_carts_user',
    2: 'latest_one_week_purchases_user',
    3: 'latest_one_week_favourites_user'
})
df = df.merge(to_merge, on='user_id', how='left')

# Count total actions by type for a given merchant
to_merge = latest_one_week_merchant['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'latest_one_week_clicks_merchant', 
    1: 'latest_one_week_carts_merchant',
    2: 'latest_one_week_purchases_merchant',
    3: 'latest_one_week_favourites_merchant'
})
df = df.merge(to_merge, on='merchant_id', how='left')

# Count total actions by type for a given pair (user, merchant)
to_merge = latest_one_week_user_merchant['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'latest_one_week_clicks_user_merchant',
    1: 'latest_one_week_carts_user_merchant',
    2: 'latest_one_week_purchases_user_merchant',
    3: 'latest_one_week_favourites_user_merchant'
})
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')


### Ratio for one week

In [22]:
groups = ['user', 'merchant', 'user_merchant']
action_types = ['clicks', 'carts', 'purchases', 'favourites']

EPSILON = 1e-8

# Compute ratios for features in user, merchant, and user-merchant categories
for group in groups:
    for feature in ['items', 'categories', 'brands', 'dates', 'periods', 'action_types']:
        # Feature ratio for each group relative to itself
        df[f'latest_one_week_{feature}_{group}_ratio'] = df[f'latest_one_week_{feature}_{group}'] / (df[f'latest_one_week_{feature}_{group}'] + EPSILON)
        df[f'latest_one_week_{feature}_{group}_ratio'] = df[f'latest_one_week_{feature}_{group}'] / (df[f'{feature}_{group}'] + EPSILON)

# Compute ratios for actions from the user perspective
for action in action_types:
    # Ratio within latest_one_week actions
    df[f'latest_one_week_{action}_user_ratio'] = df[f'latest_one_week_{action}_user'] / (
        df['latest_one_week_clicks_user'] + df['latest_one_week_carts_user'] + df['latest_one_week_purchases_user'] + df['latest_one_week_favourites_user'] + EPSILON
    )
    df[f'latest_one_week_{action}_user_ratio'] = df[f'latest_one_week_{action}_user'] / (df[f'{action}_user'] + EPSILON)

# Compute ratios for actions from the merchant perspective
for action in action_types:
    # Ratio within latest_one_week actions
    df[f'latest_one_week_{action}_merchant_ratio'] = df[f'latest_one_week_{action}_merchant'] / (
        df['latest_one_week_clicks_merchant'] + df['latest_one_week_carts_merchant'] + df['latest_one_week_purchases_merchant'] + df['latest_one_week_favourites_merchant'] + EPSILON
    )
    df[f'latest_one_week_{action}_merchant_ratio'] = df[f'latest_one_week_{action}_merchant'] / (df[f'{action}_merchant'] + EPSILON)

# Compute ratios for actions from the user-merchant perspective
for action in action_types:
    # Ratio within latest_one_week actions
    df[f'latest_one_week_{action}_user_merchant_ratio'] = df[f'latest_one_week_{action}_user_merchant'] / (
        df['latest_one_week_clicks_user_merchant'] + df['latest_one_week_carts_user_merchant'] + df['latest_one_week_purchases_user_merchant'] + df['latest_one_week_favourites_user_merchant'] + EPSILON
    )
    # Ratio of actions compared to total user-merchant actions
    df[f'latest_one_week_{action}_user_merchant_ratio'] = df[f'latest_one_week_{action}_user_merchant'] / (df[f'{action}_user_merchant'] + EPSILON)

# Ratio of actions in each merchant (user perspective)
for action in action_types:
    df[f'latest_one_week_{action}_in_merchant_ratio_perspective'] = df[f'latest_one_week_{action}_user_merchant'] / (df[f'latest_one_week_{action}_user'] + EPSILON)

# Ratio of actions in each merchant (merchant perspective)
for action in action_types:
    df[f'latest_one_week_{action}_by_user_ratio_perspective'] = df[f'latest_one_week_{action}_user_merchant'] / (df[f'latest_one_week_{action}_merchant'] + EPSILON)

# Ratio of each action type for a given user
for action in action_types:
    df[f'latest_one_week_{action}_user_ratio'] = df[f'latest_one_week_{action}_user'] / (
        df['latest_one_week_clicks_user'] + df['latest_one_week_carts_user'] + df['latest_one_week_purchases_user'] + df['latest_one_week_favourites_user'] + EPSILON
    )

# Ratio of each action type for a given merchant
for action in action_types:
    df[f'latest_one_week_{action}_merchant_ratio'] = df[f'latest_one_week_{action}_merchant'] / (
        df['latest_one_week_clicks_merchant'] + df['latest_one_week_carts_merchant'] + df['latest_one_week_purchases_merchant'] + df['latest_one_week_favourites_merchant'] + EPSILON
    )

# Ratio of each action type for a given pair (user, merchant)
for action in action_types:
    df[f'latest_one_week_{action}_user_merchant_ratio'] = df[f'latest_one_week_{action}_user_merchant'] / (
        df['latest_one_week_clicks_user_merchant'] + df['latest_one_week_carts_user_merchant'] + df['latest_one_week_purchases_user_merchant'] + df['latest_one_week_favourites_user_merchant'] + EPSILON
    )

### PCA

In [ ]:
# Fill NaN values with 0 for all columns except 'label'
df.loc[:, df.columns != 'label'] = df.loc[:, df.columns != 'label'].fillna(0)

In [24]:
pca_df = df.drop(['type', 'label'], axis=1)
pca = PCA(n_components=5)
pca.fit(pca_df)
df = df.join(pd.DataFrame(pca.transform(pca_df), index=pca_df.index).add_prefix('pca_'))

## Finishing Touch

In [28]:
df_train = df[df['type'] == 'train'].drop(['type'], axis=1)
df_test = df[df['type'] == 'test'].drop(['type', 'label'], axis=1)
X, y = df_train.drop(columns='label'), df_train['label']

In [30]:
X.to_csv("X_features.csv")

In [31]:
y.to_csv("y_features.csv")

In [35]:
X.shape[0], X.shape[1]

(1470168, 527)

In [33]:
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64